In [88]:
# read in input.txt and sample-input.txt as 2d arrays of characters

with open("../input.txt") as f:
    input = [list(line.strip()) for line in f.readlines()]
# print(input)

with open("../sample-input.txt") as f:
    sample_input = [list(line.strip()) for line in f.readlines()]
print(sample_input)

[['4', '6', '7', '.', '.', '1', '1', '4', '.', '.'], ['.', '.', '.', '*', '.', '.', '.', '.', '.', '.'], ['.', '.', '3', '5', '.', '.', '6', '3', '3', '.'], ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.'], ['6', '1', '7', '*', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '+', '.', '5', '8', '.'], ['.', '.', '5', '9', '2', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '7', '5', '5', '.'], ['.', '.', '.', '$', '.', '*', '.', '.', '.', '.'], ['.', '6', '6', '4', '.', '5', '9', '8', '.', '.']]


In [89]:
import pandas as pd
import numpy as np

# convert to pandas dataframe
df = pd.DataFrame(input)
df = df.replace(".", np.nan)
display(df)

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,NaN,NaN,2,2,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,1,1,NaN
1,NaN,NaN,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,@,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2,3,5,NaN,NaN,NaN,NaN,NaN,NaN,...,5,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,...,NaN,NaN,NaN,NaN,NaN,6,8,8,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,8,2,NaN,NaN,NaN,...,7,*,NaN,NaN,NaN,NaN,*,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,NaN,NaN,NaN,NaN,NaN,NaN,=,NaN,NaN,NaN,...,NaN,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN
136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7,6,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,NaN,NaN,NaN,NaN,NaN,NaN,*,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4,8,4,NaN,NaN


In [90]:
from itertools import groupby

block_id = 1
blocks = {}

# Function to process each row and return updated row
def process_row(row):
    global block_id
    global blocks
    # Convert the row to a string representation, handling NaNs
    str_row = row.fillna('NaN').astype(str)

    # Group contiguous numbers and non-numbers
    grouped = [list(g) for k, g in groupby(str_row, key=lambda x: x.isnumeric())]

    # Process each group
    new_row = []
    for group in grouped:
        if group[0].isnumeric():  # If the group is numeric, store in dictionary and extend with dictionary key
            block_key = f"block_{block_id}"
            blocks[block_key] = int(''.join(group))
            new_row.extend([block_key] * len(group))
            block_id += 1
        else:  # If non-numeric, just extend the group as is
            new_row.extend(group)

    # Convert back to original types, handling numeric values and NaNs
    return [float(x) if x.replace('.', '', 1).isdigit() else np.nan if x == 'NaN' else x for x in new_row]

df_processed = df.apply(process_row, axis=1, result_type='broadcast')
df_symbols = df_processed.map(lambda x: str(x) in '$%&*+-/=@#')
part_map = df_symbols.shift(-1) | df_symbols | df_symbols.shift(1)
part_map = (part_map.T.shift(-1).T | part_map | part_map.T.shift(1).T) ^ df_symbols

# display(df_processed, df_processed[part_map])

sum([ blocks[block_id] for block_id in set(df_processed[part_map].dropna(how='all').stack().reset_index(drop=True).tolist()) ])

539590